In [23]:
import yfinance as yf

In [24]:
tickers = ['AAPL', 'JPM', 'MSFT']

In [25]:
data = yf.download(tickers, start="1990-09-01", end="2024-09-01", interval="1d")

[*********************100%***********************]  3 of 3 completed


In [26]:
mydata = data['Adj Close']
mydata.index = mydata.index.tz_localize(None)

In [27]:
mydata.head()   

Ticker,AAPL,JPM,MSFT
Date,,,
1990-09-04,0.262725,2.535360,0.532032
1990-09-05,0.255625,2.592983,0.504255
1990-09-06,0.253849,2.564171,0.491435
1990-09-07,0.258288,2.636199,0.493571
1990-09-10,0.253849,2.598191,0.476479


In [28]:
import pandas as pd
import numpy as np
from fredapi import Fred
fred = Fred(api_key='dbc938743655482a0f341e0216a5c36e')

In [29]:
interest_rate = fred.get_series('FEDFUNDS', start_date='1990-09-01', end_date='2024-09-01')
unemployment_rate = fred.get_series('UNRATE', start_date='1990-09-01', end_date='2024-09-01')
interest_rate_df = interest_rate.to_frame(name='Interest_Rate')
unemployment_rate_df = unemployment_rate.to_frame(name='Unemployment_Rate')

In [30]:
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [ ]:
mydata_ret = mydata.pct_change().add_suffix('_Simple_Return')
mydata_ret

Ticker,AAPL_Simple_Return,JPM_Simple_Return,MSFT_Simple_Return
Date,,,
1990-09-04,NaN,NaN,NaN
1990-09-05,-0.027026,0.022728,-0.052209
1990-09-06,-0.006947,-0.011111,-0.025423
1990-09-07,0.017485,0.028090,0.004348
1990-09-10,-0.017184,-0.014417,-0.034631
...,...,...,...
2024-08-26,0.001499,0.003939,-0.007918
2024-08-27,0.003742,0.004608,0.000846
2024-08-28,-0.006753,0.005041,-0.007829


In [31]:
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


rsi_data = mydata.apply(lambda x: calculate_rsi(x), axis=0)
rsi_data.columns = [f"{col}_RSI" for col in rsi_data.columns]
rsi_data

,AAPL_RSI,JPM_RSI,MSFT_RSI
Date,,,
1990-09-04,NaN,NaN,NaN
1990-09-05,NaN,NaN,NaN
1990-09-06,NaN,NaN,NaN
1990-09-07,NaN,NaN,NaN
1990-09-10,NaN,NaN,NaN
...,...,...,...
2024-08-26,91.254493,95.505094,65.473267
2024-08-27,90.586240,95.702421,67.397987
2024-08-28,82.296373,95.128359,59.523133


In [32]:
ma_20 = mydata.rolling(window=20).mean().add_suffix('_20d_MA')
ma_50 = mydata.rolling(window=50).mean().add_suffix('_50d_MA')
ma_100 = mydata.rolling(window=100).mean().add_suffix('_100d_MA')

In [34]:
interest_rate_daily = interest_rate_df.reindex(mydata.index).ffill().bfill()
unemployment_rate_daily = unemployment_rate_df.reindex(mydata.index).ffill().bfill()

In [ ]:
preprocessed_daily_data = pd.concat([daily_stock_data.add_suffix('_Price'),
                                     daily_volume_data.add_suffix('_Volume'),
                                     simple_returns,
                                     rsi_data,
                                     ma_20,
                                     ma_50,
                                     ma_100,
                                     interest_rate_daily,
                                     unemployment_rate_daily], axis=1)

,Interest_Rate
Date,
1990-09-04,8.11
1990-09-05,8.11
1990-09-06,8.11
1990-09-07,8.11
1990-09-10,8.11
...,...
2024-08-26,5.33
2024-08-27,5.33
2024-08-28,5.33
